In [1]:
# desc
!pip install uuid requests pandas rdflib requests SPARQLWrapper pydantic

In [2]:
# test

In [3]:
import json
import uuid
import time
import requests

# import the smartpynector module
import smartpynector as sp

In [4]:
# Defining constants
# Knowledge engine REST API URL
# KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
KNOWLEDGE_ENGINE_URL = "http://knowledge_engine:8280/rest"
THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"

In [5]:
# Defining the Smart Connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=THERMOSTAT_API_URL,
    knowledgeBaseName="thermostat 1",
    knowledgeBaseDescription="thermostat 1",
    reasonerEnabled=False,
)

In [6]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

<Response [200]>

In [7]:
GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id ."""
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
}
HEADERS = {
    "Content-Type": "application/json",
    "knowledge-Base-Id": THERMOSTAT_API_URL,
}

In [8]:
answer_ki_id = sp.register_knowledge_interaction(
    knowledge_interaction_type="AnswerKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes=PREFIXES,
    graph_pattern=GRAPH_PATTERN,
    headers=HEADERS,
).json()["knowledgeInteractionId"]

answer_ki_id

'http://0.0.0.0:8001/thermostat/interaction/b731e424-ac32-424f-84e8-b913714356d8'

In [9]:
thermostat_actual_temperature = requests.get(
    THERMOSTAT_API_URL + "/actual_temperature"
).json()
thermostat_actual_temperature

35

In [10]:
sp.perform_answer_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=THERMOSTAT_API_URL,
    knowledge_interaction_id=answer_ki_id,
    answer_binding_set=[
        {
            "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
            "temp": f"{thermostat_actual_temperature}",
            "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
            "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
            "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
        }
    ],
)

<Response [200]>

In [11]:
# Lets try implementing a POST / REACT knowledge interaction

In [12]:
post_ki_id = sp.register_knowledge_interaction(
    "PostKnowledgeInteraction",
    KNOWLEDGE_ENGINE_URL,
    PREFIXES,
    headers=HEADERS,
    argument_graph_pattern=GRAPH_PATTERN,
).json()["knowledgeInteractionId"]
post_ki_id

'http://0.0.0.0:8001/thermostat/interaction/16f3247f-7b94-474e-9e8c-c047a4a36beb'

In [13]:
def get_temp_now():
    return requests.get(THERMOSTAT_API_URL + "/actual_temperature").json()

In [14]:
sp.perform_post_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=THERMOSTAT_API_URL,
    knowledge_interaction_id=post_ki_id, 
    argument_binding_set=[
        {
            "meas": f"<{THERMOSTAT_API_URL}/measurements/{str(uuid.uuid4())}>",
            "temp": f"{int(get_temp_now())}",
            "timestamp": f'"{sp.get_timestamp_now()}"',  # ISO 8601 format
            "room_id": f'"{THERMOSTAT_API_URL}/rooms/1"',
            "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
        }
    ]
)

    

KeyboardInterrupt: 